In [40]:
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)
import math
from random import shuffle
X_filename = "X.csv"
Y_filename = "y.csv"

GCM CODE

In [41]:
def distance(x, y, alpha):
    # Calculate psychological distance between memory examplar x and stimulus y
    dist = 0
    for i in range(len(x)):
        dist += (alpha[i] * (x[i] - y[i]))

    return abs(dist)

def similarity(dist, beta):
    # similary measure after calculating distance between x and y
    return math.exp(-beta * dist)

def get_exempler_set(dataframe, category_R):
    # Extract exemplar set dataframe of a particular label
    category_dataframe = dataframe[dataframe.label == category_R]
    return category_dataframe[["weight", 'height']]


def get_count(x, exemplar_set):
    # Calculate N(R, x)
    # count of the number of times x has been recorded as being in category R before
    values_df = exemplar_set.value_counts().reset_index()
    x_weight, x_height = x.weight, x.height
    row = values_df.loc[(values_df.weight == x_weight) & (values_df.height == x_height)]
    return row.iloc[0][0]

def exemplar_vote(y, dataframe, category_R, gamma_R, alpha, beta):
    # calculate P(R|y) without the normalization factor

    exemplar_set = get_exempler_set(dataframe, category_R)
    # print(exemplar_set)
    prob_R_y = 0 
    for x in exemplar_set.iterrows():
        x_ = x[1]
        N_R_x = get_count(x_, exemplar_set)
        distance_x_y = distance(x_, y, alpha=alpha)
        similarity_x_y = similarity(distance_x_y, beta=beta)
        prob_R_y += N_R_x * similarity_x_y
    
    prob_R_y *= gamma_R 

    return prob_R_y
    
def get_probablity(y, dataframe,categories = [1, 2, 3], alpha=[1,1], beta=1, gamma_R=[1,1,1]):
    # there are 3 category labels, 1, 2, 3,
    # we calculate P(R|y) for each category, then normalize it according to formula and 
    # return probablity for each label

    prob = []
    prob.append(exemplar_vote(y, dataframe, categories[0], gamma_R[0], alpha=alpha, beta=beta))
    prob.append(exemplar_vote(y, dataframe, categories[1], gamma_R[1], alpha=alpha, beta=beta))
    prob.append(exemplar_vote(y, dataframe, categories[2], gamma_R[2], alpha=alpha, beta=beta))

    return prob/sum(prob)

def get_output(X_df, Y_df, alpha=[1,1], beta=1, gamma_R=[1,1,1]):
    # given X_df and Y_df, calculate probablity and store output for each stimuli

    output_list = []
    for i in range(len(Y_df)):
        output_list.append(np.argmax(get_probablity(Y_df.iloc[i], X_df, alpha=alpha, beta=beta, gamma_R=gamma_R))+1)
        # probablity.append(get_probablity(Y_df.iloc[i], X_df))

    return output_list



RMC Code

In [48]:
#Utility functions:

class dLocalMAP:
    """
    See Anderson (1990, 1991)
    'Categories' renamed 'clusters' to avoid confusion.
    Discrete version.
    
    Stimulus format is a list of integers from 0 to n-1 where n is the number
    of possible features (e.g. [1,0,1])
    
    args: c, alphas
    """
    
    def __init__(self, args):
        self.partition = [[]]
        self.c, self.alpha = args
        self.alpha0 = [sum(f) for f in self.alpha]
        self.N = 0
    
    def probClustVal(self, k, i, val):
        """Find P(j|k)"""
        cj = len([x for x in self.partition[k] if x[i]==val])
        nk = len(self.partition)
        # return (cj + self.alpha[i][val])/(nk + self.alpha0[i])
        return (cj + self.alpha[i][val])/(nk + self.alpha0[i])
    
    def condclusterprob(self, stim, k):
        """Find P(F|k)"""
        pjks = []
        for i in range(len(stim)):
            cj = len([x for x in self.partition[k] if x[i]==stim[i]])
            nk = len(self.partition[k])
            # pjks.append( (cj + self.alpha[i][stim[i]])/(nk + self.alpha0[i]) )
            # print(i, self.alpha0)
            pjks.append( (cj + self.alpha[i][stim[i]])/(nk + self.alpha0[i]) )
        return np.product( pjks )
        
    
    def posterior(self, stim):
        """Find P(k|F) for each cluster"""
        pk = np.zeros( len(self.partition) )
        pFk = np.zeros( len(self.partition) )
        
        # existing clusters:
        for k in range(len(self.partition)):
            pk[k] = self.c * len(self.partition[k])/ ((1-self.c) + self.c * self.N)
            if len(self.partition[k])==0: # case of new cluster
                pk[k] = (1-self.c) / (( 1-self.c ) + self.c * self.N)
            pFk[k] = self.condclusterprob( stim, k)
        
        # put it together
        pkF = (pk*pFk) # / sum( pk*pFk )
        
        return pkF
    
    def stimulate(self, stim):
        """Argmax of P(k|F) + P(0|F)"""
        winner = np.argmax( self.posterior(stim) )
        # print( "Stim: ", stim)
        # print("Partition: ", self.partition)
        # print( self.posterior(stim))
        
        if len(self.partition[winner]) == 0:
            self.partition.append( [] )
        self.partition[winner].append(stim)
        
        self.N += 1
    
    def query(self, stimulus):
        """Queried value should be -1."""
        qdim = -1
        for i in range(len(stimulus)):
            if stimulus[i] < 0:
                if qdim != -1:
                    raise (Exception, "ERROR: Multiple dimensions queried.")
                qdim = i
        
        self.N = sum([len(x) for x in self.partition])
        
        pkF = self.posterior(stimulus)
        pkF = pkF[:-1] / sum(pkF[:-1]) # eliminate `new cluster' prob
        
        pjF = np.array( [sum( [ pkF[k] * self.probClustVal(k, qdim, j) for k in range(len(self.partition)-1)] )  for j in range(len( self.alpha[qdim] ))] )
        
        return pjF / sum(pjF)


def testlocalmapD(alpha):
    """
    Tests the Anderson's ratinal model using the Medin & Schaffer (1978) data.
    
    This script will print out the probability that each item belongs to each
    of the existing clusters or to a new cluster, and the model assign it to
    the most likely cluster. To see that the model is working correctly, you
    can follow along with Anderson (1991), which steps through in the same way.
    """
    # stims = [[1, 1, 1, 1, 1], # Medin & Schaffer (1978)
    #          [1, 0, 1, 0, 1], 
    #          [1, 0, 1, 1, 0],
    #          [0, 0, 0, 0, 0],
    #          [0, 1, 0, 1, 1],  
    #          [0, 1, 0, 0, 0]]
    # These are the classic Shepard Type II and Type IV datasets.
    # Uncomment the one you want to try out; you might want to uncomment
    # shuffling the stims too if you don't care about order.
    #stims = [[0, 0, 0, 0], [0, 0, 1, 0], [1, 1, 0, 1], [1, 1, 1, 1], [1, 0, 0, 0], [1, 0, 1, 1], [0, 1, 0, 0], [0, 1, 1, 1]] # Type IV
    #stims = [[0, 0, 0, 0], [0, 0, 1, 0], [1, 1, 0, 0], [1, 1, 1, 0], [1, 0, 0, 1], [1, 0, 1, 1], [0, 1, 0, 1], [0, 1, 1, 1]] # Type II

    stims = pd.read_csv(X_filename, names=['weight', 'height', 'label'])
    stims = stims.to_numpy()
    y_stim = pd.read_csv(Y_filename, names=['weight', 'height'])
    y_stim = y_stim.to_numpy()
    for stim in stims:
        #sprint(stim)
        stim[2] = int(stim[2]) -  1
        #print(stim[2])

    for _ in range(1):
        # alpha = [[0.05]*95, [0.05]*95, [0.5, 0.5, 0.5]]
        model = dLocalMAP([.5, alpha])
        stims = list(stims)
        shuffle(stims)
        
        for s in stims:
            model.stimulate(list(s))
        #print("Partitions:",model.partition)
        
        #print ("Prob vals for 0,0,0,0,?", model.query( [0]*(len(stims[0])-1) + [-1] ) )
        output_list = []
        for st in y_stim:
            #print(st)
            op = model.query( list(st) + [-1] )
            output_list.append(np.argmax(op) + 1)
            #print (f"Prob vals for {st} is", )

        #print ("Prob vals for 0,0,0,0,?", model.query( [85,69] + [-1] ) )
    return output_list

In [49]:
X_df = pd.read_csv(X_filename, names=['weight', 'height', 'label'])
Y_df = pd.read_csv(Y_filename, names=['weight', 'height',])

We will now try to show data that whether the models have exchangability of data or not

Case 1: GCM Model

First we will shuffle the training dataframe, and try to get label output 5 times and see whether results are effected.
Note that i chose small number 5 because code is taking long to run, but even if we increase the number, results wont change

In [58]:
g_op = []
for i in range(5):
    X_df_samp = X_df.sample(frac=1)
    output = get_output(X_df_samp, Y_df, alpha=[2,1], gamma_R=[33,40,27])
    g_op.append(output)

for idx, val in enumerate(g_op):
    print(f"{idx+1}th run:{val}")

1th run:[2, 2, 3, 2, 2, 3, 2, 2, 2, 1]
2th run:[2, 2, 3, 2, 2, 3, 2, 2, 2, 1]
3th run:[2, 2, 3, 2, 2, 3, 2, 2, 2, 1]
4th run:[2, 2, 3, 2, 2, 3, 2, 2, 2, 1]
5th run:[2, 2, 3, 2, 2, 3, 2, 2, 2, 1]


We can see that in case of GCM, for each run, after shuffling training data, output labels are not changing at all. This is because by definition, GCM doesnt has any mathematic calculation regarding this. It calculates stimuli values only after all training data are present for our case

Case 2: RMC Model

In [60]:
g_op2 = []

for i in range(5):
    X_df_samp = X_df.sample(frac=1)
    output = testlocalmapD(alpha=[[0.08]*93, [0.05]*85, [2, 1, 0.8]])
    g_op2.append(output)


for idx, val in enumerate(g_op2):
    print(f"{idx+1}th run:{val}")

1th run:[2, 2, 1, 2, 2, 2, 2, 2, 2, 1]
2th run:[2, 2, 1, 2, 2, 2, 1, 2, 2, 1]
3th run:[2, 2, 1, 2, 2, 2, 1, 2, 2, 1]
4th run:[2, 2, 1, 2, 2, 3, 2, 2, 2, 1]
5th run:[2, 2, 1, 2, 2, 3, 1, 2, 2, 1]


We can see that in case of RMC model, it doesnt assume exchangibility, i.e with each runs, depending on which order the training data enters, it has influence on test data. 

This is because prediction depends on way how the partitions are formed. Also, for prediction, we take weightaged average with a partition and its expected value. Its also mentioned in the paper "The adaptive nature of human categorization-John R Anderson" that RMC is model is order sensitive. Sensitivity will change depending on the change in values of alpha

References and Acknowledgments:
- Refered Course slides
- "The adaptive nature of human categorization-John R Anderson"
- Did some discussions with classmates : Debdeep, Sharanya